# Basic Local Alignment Search Tool (BLAST)

## What is BLAST

The Basic Local Alignment Search Tool (BLAST) is a program that can detect sequence similarity between a Query sequence and Target sequences within a database. The ability to detect sequence homology allows us to identify putative genes in a novel sequence. It also allows us to determine if a gene or a protein is related to other known genes or proteins.

BLAST is popular because it can quickly identify regions of local similarity between two sequences. More importantly, BLAST uses a robust statistical framework that can determine if the alignment between two sequences is statistically significant.

## BLAST programs

The BLAST suite consists of numerous programs which are used for various different searches:
* blastn uses a DNA query to search against a DNA database
* blastp uses a protein query to search against a protein database
* blastx uses a DNA query to search against a protein database. A BLAST search is performed using all six reading frames of the DNA query
* tblastn uses a protein query to search against all six reading frames of a DNA database
* tblastx uses all six reading frames of a DNA query to search against all six reading frames of a DNA database

## Running BLAST on the CHPC
Typically, the database used in your BLAST search would depend on the biological question you are asking. For example, if you would like to identify if an unknown gene has a homolog in humans, you would only perform a BLAST search against the human protein set. In most cases if you want to annotate a gene set, especially if you performed _de novo_ assembly, you would BLAST your sequences against a database consisting of proteins from many organisms. The NR database from the NCBI consists of about 80 000 000 protein sequences from various organisms. This will naturally take a very long time to run, especially if you have thousands of query sequences. 

The purpose of this tutorial is to show you how to speed up BLAST searches as your jobs on the CHPC only have a limited walltime before it gets terminated. Naturally you can use a smaller database than the NR, which should be a lot faster, but might also take longer than your alloted walltime. 

In this tutorial, we will use blastx to search a thousand Ginkgo biloba transcript sequences against the TAIR10 protein database. In contrast to the NR database, TAIR10 consists of 35 386 protein sequences, which will make the BLAST job a lot faster, although it will still take a couple of minutes to run.

Here is a typical BLAST script to BLAST our sequences to the TAIR10 database:

    #!/bin/bash

    #PBS -P CBBI0905
    #PBS -l select=1:ncpus=1:nodetype=haswell_reg
    #PBS -e chpc.blast.test.err
    #PBS -o	chpc.blast.test.out
    #PBS -N chpc.blast.test
    #PBS -l walltime=48:00:00
    #PBS -m ae
    #PBS -M riaan.swanepoel91@gmail.com
    #PBS -q normal

    module add chpc/BIOMODULES
    module add ncbi-blast/2.6.0
    
    cd $PBS_O_WORKDIR
    
    BLASTDB="/home/rswanepoel/lustre/data/tair10/TAIR10_pep_20101214"
    infile="/home/rswanepoel/lustre/chpc_blast_training/Gbiloba_subset.fasta"
    outfile="$(basename "${infile%.*}").tair.blastx.xml"

    blastx -evalue 1e-5 -num_alignments 20 -outfmt 5 -num_threads 1 -db ${BLASTDB} -query ${infile} -out ${outfile}
    
Running this job takes 9 minutes and 51 seconds. Naturally, an increase in the size of the BLAST database will cause a large increase in the run time of the BLAST job. 

## Speeding up BLAST

When running a BLAST job against a large database such as NR will take a long time to run and will most likely run longer than the maximum alloted walltime. However, there are various strategies we can use to speed up this BLAST job.

### Multithreading
The first method is to increase the number of CPU threads used. This method simply devides the database into equal sized chunks according to the number of threads available. Each chunk is sent to seperate threads and all query sequences is searched against each chunk of the database. 

Using this BLAST method on our dataset took 1 minute and 5 seconds.

### Query segmentation
The next method to speed up blast, is to split the query multifasta file into separate sequences and run separate BLAST searches for each sequence. 

Using this BLAST method on our dataset took 35 seconds.

### Hybrid approach
Lastly, we can combine the previous two methods by both splitting up our query sequences and using multithreading. The CHPC normal que allows us to up to 10 nodes, each with 24 CPU cores, i.e. we can use up to 240 cores for our blast job. 

The following script incorporates fault tollerence for when a BLAST job fails due to various reasons. It splits the query multifasta file into seperate fasta files in a temporary directory. The BLAST is run for each sequence using 4 cores and is afterwards zipped indicating the completion of that specific sequence. Should the BLAST job fail, the sequence will not be zipped and the BLAST job can be restarted from the failed sequence. Finally, the script concatenates all of the BLAST output into a single output file.

    #!/bin/bash
    
    #PBS -P CBBI0905
    #PBS -l select=10:ncpus=24:nodetype=haswell_reg
    #PBS -e chpc.blast.test.err
    #PBS -o chpc.blast.test.out
    #PBS -N chpc.blast.test
    #PBS -l walltime=48:00:00
    #PBS -m ae
    #PBS -M riaan.swanepoel91@gmail.com
    #PBS -q normal
    
    module add chpc/BIOMODULES
    module add ncbi-blast/2.6.0
    module add chpc/gnu/parallel-20160422
    
    export PARALLEL="--env PATH --env LD_LIBRARY_PATH --env LOADEDMODULES --env _LMFILES_ --env MODULE_VERSION --env MODULEPATH --env MODULEVERSION_STACK --env MODULESHOME"
    
    workdir="$PBS_O_WORKDIR"
    ID_FMT="%01d"
    
    split_prefix="blasttest_subset"
    BLASTCMD=$(which blastx)
    BLASTARGS="-evalue 1e-5 -num_alignments 20 -outfmt 5 -num_threads 4"
    BLASTDB="/home/rswanepoel/lustre/data/tair10/TAIR10_pep_20101214"
    infile="/home/rswanepoel/lustre/chpc_blast_training/Gbiloba_subset.fasta"
    outfile="$(basename "${infile%.*}").tair.blastx.xml"
    
    cd $PBS_O_WORKDIR
    
    mkdir tmp
    mkdir tmp/done
    
    cat ${infile} | csplit -z -f ${workdir}/tmp/${split_prefix} -b "${ID_FMT}.split.fasta" - '/^>/' '{*}'
    
    ls tmp/blasttest*.fasta | parallel -M -j 6 -u --sshloginfile ${PBS_NODEFILE} "cd ${PBS_O_WORKDIR}; ${BLASTCMD} -db ${BLASTDB} -query {} ${BLASTARGS} -out {}.xml && gzip --best {} {}.xml"
    
    mv tmp/*.fasta.gz tmp/done/
    zcat tmp/*.xml.gz > ${outfile}
    
This BLAST job took 19 seconds to complete. 

While the length of the BLAST job will depend on the number of sequences and the size of the database, the fault tollerence used in this script acts as a "checkpoint" should your job run longer than the alloted walltime. 